In [35]:
import pandas as pd
import sklearn

sklearn.set_config(transform_output="pandas")

df = pd.read_csv("recs2020_public_v7.csv")
df = df[df["KWH"] < 6000]
df = df[df["KWH"] > 1000]
base_cols = ["NCOMBATH", "TOTROOMS", "NHSLDMEM", "ATHOME", "KWH"]
device_cols = [
    "TVCOLOR",
    "OVEN",
    "COOKTOP",
    "MICRO",
    "ZFREEZER",
    "DESKTOP",
    "NUMLAPTOP",
    "DVD"
]
print(df.count())
device_cols_in_df = [col for col in device_cols if col in df.columns]
df["NUM_DEVICES"] = df[device_cols_in_df].fillna(0).sum(axis=1)
model_df = df[base_cols + ["NUM_DEVICES"]]

features = ["NCOMBATH", "TOTROOMS", "NHSLDMEM", "ATHOME", "NUM_DEVICES"]

model_df.head()

DOEID           4689
REGIONC         4689
DIVISION        4689
STATE_FIPS      4689
state_postal    4689
                ... 
EVCHRGDLR       4680
EVCHRGHWY       4679
EVCHRGOTH       4681
EVHOMEAMT       4685
EVCHRGTYPE      4686
Length: 799, dtype: int64


,NCOMBATH,TOTROOMS,NHSLDMEM,ATHOME,KWH,NUM_DEVICES
1,1,3,1,5,5243.05,3
2,1,4,1,3,2387.64,0
4,1,3,2,5,5869.70,4
8,1,3,1,5,3969.96,2
9,1,4,1,0,3632.40,2


In [36]:
from sklearn.model_selection import train_test_split

y = model_df["KWH"]
df = model_df.drop(columns=["KWH"])

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.3, random_state=0
)

In [37]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

def evaluate_model(
    y_train,
    y_test,
    y_pred_train,
    y_pred_test
) -> None:
    rmse_train = root_mean_squared_error(y_train, y_pred_train)
    mae_train = mean_absolute_error(y_train, y_pred_train)

    rmse_test = root_mean_squared_error(y_test, y_pred_test)
    mae_test = mean_absolute_error(y_test, y_pred_test)

    print(f"Train RMSE {rmse_train:.2f}, MAE: {mae_train:.2f}")
    print(f"Test  RMSE {rmse_test:.2f}, MAE: {mae_test:.2f}")

rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)

y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)
evaluate_model(y_train, y_test, y_pred_train, y_pred_test)

Train RMSE 945.59, MAE: 742.57
Test  RMSE 1312.61, MAE: 1071.85


In [38]:
new_data = pd.DataFrame({
    "NCOMBATH": [0.5],
    "TOTROOMS": [0.2],
    "NHSLDMEM": [0.5],
    "ATHOME": [0.5],
    "NUM_DEVICES": [0.14]
})
predicted_kwh = rf.predict(new_data)
print(f"Przewidywane zużycie energii (KWH): {predicted_kwh[0]:.2f}")

Przewidywane zużycie energii (KWH): 3028.54
